# Low-level Data Access
{{frescox}} simulations can often produce more than one output files containing simulation results.  In addition, the same results can be provided in more than one of those files.  Each of the {{bfrescox}} `parse_*` data access functions, whose use are demonstrated in the examples, are implemented to access the most precise version of the particular observables that they access.

The one exception to this rule is the `parse_fort16` function, which is a lower-level data access function that can be used to access different data quantities stored in the `fort.16` output file.  In particular, it might provide access to data not available through the other `parse_*` functions.

This example uses demonstrates a simple use of this function and includes a cautionary tale.

## Generate Elastic Scattering Output

Note that in this example, we specifically request that all {{frescox}} inputs, intermediate files, and outputs for this single simulation be written to the new, dedicated folder ``MY_RESULTS``.

**TODO**: I can understand that I have to explicitly state that I want the template file in my results folder since it's generated by a different command and the template is an intermediate result that users create before running Frescox.  In particular, it's not an input or output.  However it looks like I also have to specifically add `MY_RESULTS` to the output filename, which is not what I expect.  If you're asking me to specify the cwd, why not put all output there?
* One way to think about this is that we would pass to a `Results` class constructor the results folder rather than the names to each output file that we know how to deal with.  In that sense, the folder is *THE* Frescox file, including its input configuration.
* Thoughts?

In [1]:
# Hide this cell in the book rendering

import os
import shutil

import numpy as np

from pathlib import Path

# Make sure that this is the same folder name to which results are
# written below.
RESULTS_FOLDER = Path.cwd().joinpath("ElasticResults")

# To ensure that our filename output is correct, ensure that the output folder
# does not exist before running simulation.  This is also a nice test of the
# run_simulation() functionality.
if RESULTS_FOLDER.exists():
    assert RESULTS_FOLDER.is_dir()
    shutil.rmtree(RESULTS_FOLDER)
assert not RESULTS_FOLDER.exists()

In [2]:
import bfrescox

MY_RESULTS = Path.cwd().joinpath("ElasticResults")
TEMPLATE_NAME = MY_RESULTS.joinpath("78Ni_p_elastic_example.template")
FRESCOX_OUTPUT = MY_RESULTS.joinpath("78Ni_p_elastic_example.out")

# Conservative check to avoid accidentally overwriting a results folder
assert not MY_RESULTS.exists()
os.mkdir(MY_RESULTS)

configuration_template_elastic = {
    "reaction_name": "p+Ni78 Coulomb + Nuclear",
    "target_mass_amu":     78, "target_atomic_number":     28, "target_spin" :     0,
    "projectile_mass_amu":  1, "projectile_atomic_number":  1, "projectile_spin" : 0.5,
    "J_tot_min": 0.0, "J_tot_max": 60.0,
    "E_lab_MeV":    50.0,
    "R_match_fm":    1.2,
    "E_0_MeV":      60.0,
    "step_size_fm":  0.1
}
parameters_elastic = {
    "V":  40.0, "r":   1.2, "a":   0.65, "W":   10.0, "rw":   1.2, "aw":   0.5,
    "Vs":  0,   "rs":  1.2, "as":  0.5,  "Ws":   0,   "rws":  1.2, "aws":  0.5,
    "Vso": 0,   "rso": 1.0, "aso": 0.65, "Wso":  0,   "rwso": 1.0, "awso": 0.65,
    "rC": 1.2
}

bfrescox.generate_elastic_template(
    TEMPLATE_NAME, **configuration_template_elastic
)
cfg = bfrescox.Configuration.from_template(
    TEMPLATE_NAME,
    "78Ni_p_elastic_example.in",
    parameters_elastic,
    overwrite=True
)
bfrescox.run_simulation(cfg, FRESCOX_OUTPUT, cwd=MY_RESULTS, overwrite=True)

We expect to see at least
* Our template file with parameter values still missing
* Our final input file with the generic `frescox.in` filename
* The ``FRESCOX_OUTPUT`` that captured results written to standard output by {{frescox}}.

However, in the following list of the contents of ``MY_RESULTS`` we some that indeed other output files were created.

In [3]:
# Remove this cell from book rendering

# Confirm that run_simulation wrote to the expected location
assert RESULTS_FOLDER == MY_RESULTS
contents = list(MY_RESULTS.glob("*"))
assert contents
assert TEMPLATE_NAME.is_file()
assert MY_RESULTS.joinpath(FRESCOX_OUTPUT) in contents
expected = {3, 4, 7, 13, 16, 17, 34, 35, 38, 39, 40, 45, 48, 56, 89, 156, 201, 239}
fort_all = set([int(str(f.name).strip("fort.")) for f in contents if str(f.name).startswith("fort.")])
assert fort_all == expected

print()
for each in contents:
    print(each.name)


fort.39
fort.38
fort.40
fort.3
fort.4
fort.13
fort.48
fort.156
frescox.in
78Ni_p_elastic_example.template
fort.201
fort.239
fort.56
fort.34
fort.35
fort.7
fort.17
fort.16
fort.45
fort.89
78Ni_p_elastic_example.out


## Load Observable from Two Files
For standard observables, users would typically prefer to load results directly using the observable-specific `parse_*` data access functions.  In this case, we'll load cross section data.


In [4]:
r2r_df = bfrescox.parse_differential_xs.ratio_to_rutherford(FRESCOX_OUTPUT)

We know that this data is also stored in the `fort.16` file, so we can also access the data using the general-use low-level function.

In [5]:
fort16 = bfrescox.parse_fort16(MY_RESULTS.joinpath("fort.16"))
r2r_fort16_df = fort16["channel_1"]
r2r_fort16_df.index = r2r_fort16_df.Theta

# The final angle is slightly different between the two, so ignore for now.
assert len(r2r_fort16_df) == len(r2r_df)
assert all(r2r_fort16_df.index[:-1] == r2r_df.index[:-1])
assert r2r_fort16_df.index[-1] == r2r_df.index[-1] + 0.01

r2r_df.drop(labels=r2r_df.index[-1], axis=0, inplace=True)
r2r_df["fort_16"] = r2r_fort16_df.sigma.iloc[:-1]
display(r2r_df)

,differential_xs_ratio_to_rutherford,fort_16
angle_degrees,,
0.01,1.000000,1.0000
1.00,0.999875,0.9999
2.00,1.000286,1.0000
3.00,1.001543,1.0020
4.00,1.003480,1.0030
...,...,...
175.00,0.410368,0.4104
176.00,0.411381,0.4114
177.00,0.412170,0.4122


Note that the two versions of the same simulation results are indeed different.  For this particular case, a review of the contents of `fort.16` and `FRESCOX_OUTPUT` show that the results accessed through `parse_differential_xs` were loaded from `FRESCOX_OUTPUT`, which records this data with more decimal digits than found in `fort.16`.

**TODO**: Why isn't `diff` all zeros?!

In [6]:
# Hide this cell in book rendering

# Confirm differences
diff = np.fabs(r2r_df.differential_xs_ratio_to_rutherford - r2r_df.fort_16).values
assert diff[0] == 0.0
assert all(diff[1:] > 0.0)

rounded = r2r_df.differential_xs_ratio_to_rutherford.round(4).values
assert any(rounded != r2r_df.differential_xs_ratio_to_rutherford)
#assert all(rounded == r2r_df.fort_16)
r2r_df["rnd_diff"] = r2r_df.fort_16 - rounded
r2r_df[r2r_df.rnd_diff != 0.0]

,differential_xs_ratio_to_rutherford,fort_16,rnd_diff
angle_degrees,,,
2.0,1.000286,1.00000,-0.00030
3.0,1.001543,1.00200,0.00050
4.0,1.003480,1.00300,-0.00050
5.0,1.005858,1.00600,0.00010
6.0,1.008452,1.00800,-0.00050
7.0,1.011064,1.01100,-0.00010
8.0,1.013526,1.01400,0.00050
9.0,1.015700,1.01600,0.00030
10.0,1.017470,1.01700,-0.00050
